In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [94]:
nt = 50           # number of temperature points
N=10              # size of lattice, N x N
eqsteps = 2000    # number of MC sweeps for equilibration
mcsteps = 2500    # number of MC sweeps for calculation

T = np.linspace(1.53, 3.28, nt)
Energy, Magnetization, SpecificHeat, Susceptibility = np.zeros(nt), np.zeros(nt), np.zeros(nt), np.zeros(nt)
Beta = 1/T

In [96]:
state = 2*np.random.randint(2,size=(N,N))-1 
print(state) 

[[-1  1 -1  1  1  1 -1  1  1 -1]
 [-1  1 -1  1 -1  1 -1 -1  1 -1]
 [ 1 -1 -1  1  1  1  1  1  1 -1]
 [-1  1 -1 -1  1  1 -1  1 -1  1]
 [ 1 -1  1 -1  1  1  1 -1 -1 -1]
 [-1  1 -1 -1  1  1 -1 -1  1 -1]
 [-1 -1 -1 -1 -1  1  1  1 -1  1]
 [-1 -1 -1 -1 -1 -1 -1  1  1  1]
 [-1  1 -1  1  1 -1  1  1  1 -1]
 [ 1  1 -1  1 -1 -1  1  1  1 -1]]


In [109]:
a = np.random.randint(0, N)
b = np.random.randint(0, N)
s =  state[a, b]
nb = state[(a+1)%N,b] + state[(a-1)%N,b] + state[a,(b+1)%N] + state[a,(b-1)%N]

In [110]:
delta_E = 2*s*nb
print(delta_E)
print(delta_E*Beta)

4
[2.61437908 2.55474453 2.49776985 2.44328098 2.3911187  2.34113712
 2.29320229 2.24719101 2.20298977 2.16049383 2.11960636 2.08023774
 2.04230489 2.00573066 1.97044335 1.93637621 1.90346703 1.87165775
 1.84089415 1.81112549 1.78230426 1.75438596 1.72732881 1.70109356
 1.67564333 1.6509434  1.62696107 1.60366552 1.58102767 1.55902004
 1.53761669 1.51679307 1.49652592 1.47679325 1.45757418 1.43884892
 1.42059868 1.40280561 1.38545275 1.36852395 1.35200386 1.33587786
 1.32013201 1.30475303 1.28972824 1.27504554 1.26069338 1.24666073
 1.23293703 1.2195122 ]
